YT Vid: https://www.youtube.com/watch?v=cO5g5qLrLSo

# Import Libs

In [2]:
import random
import gym

In [3]:
env = gym.make("CartPole-v0")
states = env.observation_space.shape[0]
actions = env.action_space.n

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\gym\envs\registration.py:505: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1` with the environment ID `CartPole-v1`.
  logger.warn(


In [4]:
states

4

In [30]:
dir(env)

['__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_action_space',
 '_elapsed_steps',
 '_is_protocol',
 '_max_episode_steps',
 '_metadata',
 '_np_random',
 '_observation_space',
 '_reward_range',
 'action_space',
 'class_name',
 'close',
 'env',
 'metadata',
 'np_random',
 'observation_space',
 'render',
 'reset',
 'reward_range',
 'seed',
 'spec',
 'step',
 'unwrapped']

In [5]:
actions

2

### Random actions

In [6]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0
    
    while not done:
        env.render()
        action = random.choice([0, 1])
        n_state, reward, done, info = env.step(action)
        score += reward
    print(f"Episode {episode} Score: {score}")

Episode 1 Score: 23.0
Episode 2 Score: 27.0
Episode 3 Score: 23.0
Episode 4 Score: 10.0
Episode 5 Score: 15.0
Episode 6 Score: 29.0
Episode 7 Score: 39.0
Episode 8 Score: 10.0
Episode 9 Score: 15.0
Episode 10 Score: 29.0


## Deep Learning Model with reinforcement learning

In [6]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [14]:
def build_model(states, actions):
    model = Sequential()
    model.add(Flatten(input_shape=(1,states)))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(24, activation="relu"))
    model.add(Dense(actions, activation="linear"))
    return model

In [15]:
model = build_model(states, actions)

In [16]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 4)                 0         
                                                                 
 dense_3 (Dense)             (None, 24)                120       
                                                                 
 dense_4 (Dense)             (None, 24)                600       
                                                                 
 dense_5 (Dense)             (None, 2)                 50        
                                                                 
Total params: 770
Trainable params: 770
Non-trainable params: 0
_________________________________________________________________


## Reinforcementlearning agent

In [1]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\GitHub\Nachzeichner-KI\env\lib\site-packages\IPython\core\interactiveshell.py", line 3369, in run_code
  File "C:\Users\wasse\AppData\Local\Temp\ipykernel_23456\2148453933.py", line 1, in <cell line: 1>
    from rl.agents import DQNAgent
ModuleNotFoundError: No module named 'rl'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\GitHub\Nachzeichner-KI\env\lib\site-packages\pygments\styles\__init__.py", line 78, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\GitHub\Nachzeichner-KI\env\lib\site-packages\IPython\core\interactiveshell.py", line 1982, in showtraceback
  File "c:\GitHub\Nachzeichner-KI\env\lib\site-packages\IPython\core\ultratb.py", line 1118, in structured_traceback
  File "c:\GitHub\Nachzeichner-KI\env\lib\site-pa

In [31]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    
    return dqn

^^ Erstellt einen Deep Q Network Agent (wahrscheinlich Deep Q-Learning) (DQN).

Dabei wird die BoltzmannQPolicy verwendet, als Regelset für die Entscheidung, welche Action genutzt werden soll.

Die Memory ist wahrscheinlich ein Speichercontainer für den Agent. Die Agents können angeblich sehr viel RAM fressen.

In [32]:
dqn = build_agent(model, actions)
dqn.compile(Adam(lr=1e-3), metrics=["mae"])
dqn.fit(env, nb_steps=1000, visualize=False, verbose = 1)

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Training for 1000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 16:35 - reward: 1.0000

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\keras\engine\training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 10 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


   21/10000 [..............................] - ETA: 5:42 - reward: 1.0000

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 11 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 12 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 13 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 14 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please

   31/10000 [..............................] - ETA: 4:51 - reward: 1.0000

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 20 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 21 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 22 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 23 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please

   43/10000 [..............................] - ETA: 4:10 - reward: 1.0000

c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 30 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)
c:\GitHub\Nachzeichner-KI\env\lib\site-packages\rl\memory.py:38: DeprecationWarning: This function is deprecated. Please call randint(1, 31 + 1) instead
  batch_idxs = np.random.random_integers(low, high - 1, size=size)


  997/10000 [=>............................] - ETA: 1:57 - reward: 1.0000done, took 13.150 seconds


In [33]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history["episode_reward"]))

Testing for 100 episodes ...
Episode 1: reward: 200.000, steps: 200
Episode 2: reward: 175.000, steps: 175
Episode 3: reward: 155.000, steps: 155
Episode 4: reward: 171.000, steps: 171
Episode 5: reward: 175.000, steps: 175
Episode 6: reward: 169.000, steps: 169
Episode 7: reward: 166.000, steps: 166
Episode 8: reward: 165.000, steps: 165
Episode 9: reward: 183.000, steps: 183
Episode 10: reward: 194.000, steps: 194
Episode 11: reward: 197.000, steps: 197
Episode 12: reward: 200.000, steps: 200
Episode 13: reward: 185.000, steps: 185
Episode 14: reward: 200.000, steps: 200
Episode 15: reward: 164.000, steps: 164
Episode 16: reward: 186.000, steps: 186
Episode 17: reward: 200.000, steps: 200
Episode 18: reward: 178.000, steps: 178
Episode 19: reward: 200.000, steps: 200
Episode 20: reward: 200.000, steps: 200
Episode 21: reward: 157.000, steps: 157
Episode 22: reward: 161.000, steps: 161
Episode 23: reward: 198.000, steps: 198
Episode 24: reward: 179.000, steps: 179
Episode 25: reward: 

In [35]:
_ = dqn.test(env, nb_episodes=1, visualize=True)

Testing for 1 episodes ...
Episode 1: reward: 157.000, steps: 157
